In [76]:
import random
from functools import reduce

class Verifier:
    
    def __init__(self):
        # questions are ex: [1, 1] (first row and first column)
        self.questions = [[x, y] for x in [0, 1, 2] for y in [0, 1, 2]]
    
    # randomly sample one question
    def sample_question(self):
        return random.choice(self.questions)
    
    def query_alice(self, question):
        return question[0]
    
    def query_bob(self, question):
        return question[1]
    
    # check if the winning condition is satisfied.
    def judge(self, question, a, b):
        
        # parity of alice = 0, parity of bob = 1, intersection of alice and bob agrees
        return parity(a) == 0 and parity(b) == 1 and a[question[1]] == b[question[0]]
    
def parity(x):
    # assume x is always a list of 3 binary bits
    return reduce((lambda i, j: ( i + j) % 2), x)



In [77]:
v = Verifier()
print(v.questions)
v.judge([1, 0], [1, 1, 0], [1, 1, 1])

[[0, 0], [0, 1], [0, 2], [1, 0], [1, 1], [1, 2], [2, 0], [2, 1], [2, 2]]


True

In [78]:
class Alice:
    
    """
    initialize player with their shared square
    square is a 2D list to represent a 3 x 3 bit matrix
    """
    def __init__(self, square):
        self.square = square
    
    """
    Alice simply ouptut her row
    if there is a special char '*'
    she replace '*' with a bit that satisfy the parity condition
    """ 
    def answer(self, question):
        a = self.square[question].copy()
        if '*' not in a:
            return a
        else:
            i = a.index('*')
            a[i] = (sum(a[:i]) + sum(a[i+1:]) )% 2
            return a
                    
                

class Bob:
    """
    initialize player with their shared square
    square is a 2D list to represent a 3 x 3 bit matrix
    """
    def __init__(self, square):
        self.square = square
    
    """
    Bob output the column
    if there is a special char '*'
    he replace '*' with a bit that satisfy the parity condition
    """ 
    def answer(self, question):
        b = []
        for row in self.square:
            b.append(row[question])
        if '*' not in b:
            return b
        else:
            i = b.index('*')
            b[i] = (sum(b[:i]) + sum(b[i+1:]) + 1) % 2
            return b

    
# test run:
# define square
square = [[1, 0, 1], [0, 1, 1], [0, 0, '*']]
alice = Alice(square)
bob = Bob(square)
v = Verifier()

for q in v.questions:
    a = alice.answer(q[0])
    b = bob.answer(q[1])
    print(q, a, b, v.judge(q, a, b))

[0, 0] [1, 0, 1] [1, 0, 0] True
[0, 1] [1, 0, 1] [0, 1, 0] True
[0, 2] [1, 0, 1] [1, 1, 1] True
[1, 0] [0, 1, 1] [1, 0, 0] True
[1, 1] [0, 1, 1] [0, 1, 0] True
[1, 2] [0, 1, 1] [1, 1, 1] True
[2, 0] [0, 0, 0] [1, 0, 0] True
[2, 1] [0, 0, 0] [0, 1, 0] True
[2, 2] [0, 0, 0] [1, 1, 1] False


In [85]:
# let's find a deterministic strategy that can win 8/9
# let's enumerate all squares
def get_squares():
    x = [0, 1]
    return [[[a, b, c], [d, e, f], [g, h, i]]
           for a in x for b in x for c in x
           for d in x for e in x for f in x
           for g in x for h in x for i in x]

# replace each square in the position with '*'
def replace_squares(row, col, squares):
    for s in squares:
        s[row][col] = '*'
    replaced = []
    for s in squares:
        if s not in replaced:
            replaced.append(s)
    return replaced
    
    
def get_optimal(row, col):
    squares = get_squares()
    squares = replace_squares(row, col, squares)

    optimal_square = []
    v = Verifier()
    for s in squares:
        correct = 0
        alice = Alice(s)
        bob = Bob(s)
        for q in v.questions:
            a = alice.answer(q[0])
            b = bob.answer(q[1])
            if v.judge(q, a, b):
                correct = correct + 1
        if correct == 8:
            optimal_square.append(s)
    return optimal_square


optimal_strategies = {}
for r in [0, 1, 2]:
    for c in [0, 1, 2]:
        optimal_strategies['({}, {})'.format(r, c)] = get_optimal(r, c)

print("The number of optimal deterministic strategies in each exclusion set is:")
print([len(optimal_strategies['({}, {})'.format(r, c)]) for r in [0, 1, 2] for c in [0, 1, 2]])
print("There are in total {} optmal strategies out of {} squares".format(16 * 9, 256 * 9))
        
for x, es in optimal_strategies.items():
    print(x)
    for s in es:
        print(s)

The number of optimal deterministic strategies in each exclusion set is:
[16, 16, 16, 16, 16, 16, 16, 16, 16]
There are in total 144 optmal strategies out of 2304 squares
(0, 0)
[['*', 0, 0], [0, 0, 0], [0, 1, 1]]
[['*', 0, 0], [0, 1, 1], [0, 0, 0]]
[['*', 0, 0], [1, 0, 1], [1, 1, 0]]
[['*', 0, 0], [1, 1, 0], [1, 0, 1]]
[['*', 0, 1], [0, 0, 0], [1, 1, 0]]
[['*', 0, 1], [0, 1, 1], [1, 0, 1]]
[['*', 0, 1], [1, 0, 1], [0, 1, 1]]
[['*', 0, 1], [1, 1, 0], [0, 0, 0]]
[['*', 1, 0], [0, 0, 0], [1, 0, 1]]
[['*', 1, 0], [0, 1, 1], [1, 1, 0]]
[['*', 1, 0], [1, 0, 1], [0, 0, 0]]
[['*', 1, 0], [1, 1, 0], [0, 1, 1]]
[['*', 1, 1], [0, 0, 0], [0, 0, 0]]
[['*', 1, 1], [0, 1, 1], [0, 1, 1]]
[['*', 1, 1], [1, 0, 1], [1, 0, 1]]
[['*', 1, 1], [1, 1, 0], [1, 1, 0]]
(0, 1)
[[0, '*', 0], [0, 0, 0], [1, 0, 1]]
[[0, '*', 0], [0, 1, 1], [1, 1, 0]]
[[0, '*', 0], [1, 0, 1], [0, 0, 0]]
[[0, '*', 0], [1, 1, 0], [0, 1, 1]]
[[0, '*', 1], [0, 0, 0], [1, 1, 0]]
[[0, '*', 1], [0, 1, 1], [1, 0, 1]]
[[0, '*', 1], [1, 0, 1]